In [8]:
# Task1
from queue import PriorityQueue

class Agent:
    def __init__(self, graph, start, goal):
        self.graph = graph
        self.start = start
        self.goal = goal

    def search(self):
        pass

# Goal-Based Agent for DFS
class Depth_first_search(Agent):
    def search(self):
        stack = [(self.start, [self.start])]
        while stack:
            (node, path) = stack.pop()
            if node == self.goal:
                return path
            for neighbor in reversed(self.graph.get(node, [])):
                if neighbor not in path:
                    stack.append((neighbor, path + [neighbor]))
        return None

# Goal-Based Agent for DLS
class Depth_limited_search(Agent):
    def __init__(self, graph, start, goal, depth_limit):
        super().__init__(graph, start, goal)
        self.depth_limit = depth_limit

    def search(self):
        def dls(node, path, depth):
            if depth > self.depth_limit:
                return None
            if node == self.goal:
                return path
            for neighbor in self.graph.get(node, []):
                if neighbor not in path:
                    result = dls(neighbor, path + [neighbor], depth + 1)
                    if result:
                        return result
            return None

        return dls(self.start, [self.start], 0)

# Utility-Based Agent for UCS
class Uniform_cost_search(Agent):
    def search(self):
        pq = PriorityQueue()
        pq.put((0, self.start, [self.start]))
        visited = {}

        while not pq.empty():
            cost, node, path = pq.get()
            if node in visited and visited[node] <= cost:
                continue
            visited[node] = cost

            if node == self.goal:
                return path, cost

            for neighbor, weight in self.graph.get(node, {}).items():
                if neighbor not in path:
                    pq.put((cost + weight, neighbor, path + [neighbor]))

        return None

graph = {
    'A': {'B': 2, 'C': 9},
    'B': {'A': 2, 'D': 7, 'E': 5},
    'C': {'A': 9, 'F': 3},
    'D': {'B': 2},
    'E': {'B': 5, 'F': 1},
    'F': {'C': 3, 'E': 1}
}

dfs_agent = Depth_first_search(graph, 'A', 'D')
print("DFS Path:", dfs_agent.search())

dls_agent = Depth_limited_search(graph, 'A', 'F', 2)
print("DLS Path:", dls_agent.search())

ucs_agent = Uniform_cost_search(graph, 'A', 'C')
print("UCS Path and Cost:", ucs_agent.search())


DFS Path: ['A', 'B', 'D']
DLS Path: ['A', 'C', 'F']
UCS Path and Cost: (['A', 'C'], 9)


In [14]:
# Task2
distances = [
    [0, 10, 15, 20],
    [10, 0, 5, 25],
    [15, 5, 0, 40],
    [20, 25, 40, 0]
]

def travelling_salesman(current_city, visited, num_cities, path, min_path):

    if sum(visited) == num_cities:
        path.append(0)  # Return to starting city
        return distances[current_city][0], path

    min_cost = float('inf')  # Start with a large cost
    best_path = None

    # Try visiting each city
    for next_city in range(num_cities):
        if not visited[next_city]:  # If city is not visited
            visited[next_city] = 1  # Mark as visited
            new_path = path + [next_city]  # Add to path
            cost, temp_path = tsp(next_city, visited, num_cities, new_path, min_path)
            total_cost = distances[current_city][next_city] + cost

            if total_cost < min_cost:  # Keep track of minimum cost
                min_cost = total_cost
                best_path = temp_path  # Store the best path

            visited[next_city] = 0  # Backtrack (unmark the city)

    return min_cost, best_path


num_cities = len(distances)
visited = [0] * num_cities
visited[0] = 1
optimal_cost, best_path = travelling_salesman(0, visited, num_cities, [0], [])

city_labels = ['A', 'B', 'C', 'D']
best_path_labels = [city_labels[i] for i in best_path]

print(f"Optimal Cost: {optimal_cost}")
print(f"Optimal Path: {' -> '.join(best_path_labels)}")


Optimal Cost: 65
Optimal Path: A -> C -> B -> D -> A


In [16]:
# Task 3
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': ['H'],
    'E': [],
    'F': ['I'],
    'G': [],
    'H': ['G'],
    'I': []
}


def dls(node, goal, depth):
    if depth == 0:
        return [node] if node == goal else None
    if depth > 0:
        for neighbor in graph.get(node, []):
            path = dls(neighbor, goal, depth - 1)
            if path:
                return [node] + path
    return None

def id_dfs(start, goal, max_depth):
    for depth in range(max_depth):
        path = dls(start, goal, depth)
        if path:
            return path
    return None

def bidirectional_search(graph, start, goal):
    if start == goal:
        return [start]

    forward_queue = [start]
    backward_queue = [goal]
    forward_visited = {start: None}
    backward_visited = {goal: None}

    while forward_queue and backward_queue:
        if expand_search(forward_queue, forward_visited, backward_visited):
            return construct_path(forward_visited, backward_visited)
        if expand_search(backward_queue, backward_visited, forward_visited):
            return construct_path(forward_visited, backward_visited)

    return None

def expand_search(queue, visited, opposite_visited):
    current = queue.pop(0)
    for neighbor in graph.get(current, []):
        if neighbor not in visited:
            visited[neighbor] = current
            queue.append(neighbor)
            if neighbor in opposite_visited:
                return True
    return False

def construct_path(forward_visited, backward_visited):
    meeting_point = set(forward_visited.keys()) & set(backward_visited.keys())
    if not meeting_point:
        return None
    meeting_point = meeting_point.pop()

    path = []
    node = meeting_point
    while node is not None:
        path.append(node)
        node = forward_visited[node]
    path.reverse()

    node = backward_visited[meeting_point]
    while node is not None:
        path.append(node)
        node = backward_visited[node]

    return path

# Running IDDFS
start, goal = 'A', 'G'
max_depth = 5
print("IDDFS Path:", id_dfs(start, goal, max_depth))

# Running Bidirectional Search
print("Bidirectional Search Path:", bidirectional_search(graph, start, goal))


IDDFS Path: ['A', 'C', 'G']
Bidirectional Search Path: None
